## Question 2 : Poisson Blending
In this question, we blend an image into another image using Poisson blending
method.
Target and source images both have n pixels, and we also have a mask. We create a vector matrix b
with n elements, each element corresponding to one pixel in target.
If a pixel is in the mask, its corresponding element in b will be the Laplacian of
source in that pixel. Otherwise, it will be the value of that pixel in target.
After creating b, we also create an $n \times n$ sparse matrix $A$,
which is the coefficient matrix of the equations used for computing final image.
Since $A$ is sparse, we use sparse matrix data structure implemented in `scipy.sparse` for
working with it.
Finally, we solve the least square error problem $Af=b$. Result $n \times 1$ vector
$f$ contains the value of each pixel in the blended image.
Functions are implemented in `q2_funcs.py`, and main code of the problem is
in `q2.py`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.sparse.linalg as sparse_la
from scipy import sparse


First function simply computes the Laplacian of an image:

In [ ]:
def get_laplacian(img):
    kernel = np.asarray([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], dtype='float32')
    result = cv2.filter2D(img, ddepth=-1, kernel=kernel)
    return result


Function `generate_matrix_b` creates matrix $b$, just in the way explained in the begining:



In [ ]:
def generate_matrix_b(source, target, mask):
    source_laplacian_flatten = get_laplacian(source).flatten('C')
    target_flatten = target.flatten('C')
    mask_flatten = mask.flatten('C')
    b = (mask_flatten) * source_laplacian_flatten + (1 - mask_flatten) * target_flatten
    return b


`generate_matrix_A` creates coefficient matrix $A$:

In [ ]:
def generate_matrix_A(mask):
    data, cols, rows = [], [], []
    h, w = mask.shape[0], mask.shape[1]
    mask_flatten = mask.flatten('C')
    zeros = np.where(mask_flatten == 0)
    ones = np.where(mask_flatten == 1)
    # adding ones to data
    n = zeros[0].size
    data.extend(np.ones(n, dtype='float32').tolist())
    rows.extend(zeros[0].tolist())
    cols.extend(zeros[0].tolist())

    # adding 4s to data
    m = ones[0].size
    data.extend((np.ones(m, dtype='float32') * (4)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend(ones[0].tolist())

    # adding -1s
    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] - 1).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] + 1).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] - w).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] + w).tolist())
    return data, cols, rows

Next function solves the LSE problem $Af=b$ and returns $f$

In [ ]:
def solve_sparse_linear_equation(data, cols, rows, b, h, w):
    sparse_matrix = sparse.csc_matrix((data, (rows, cols)), shape=(h * w, h * w), dtype='float32')
    f = sparse_la.spsolve(sparse_matrix, b)
    f = np.reshape(f, (h, w)).astype('float32')
    return f

Final function `blend_images` takes source,target and mask as input and applies blending, using above functions :

In [ ]:
def blend_images(source, target, mask):
    h, w = source.shape[0], source.shape[1]
    source_b, source_g, source_r = cv2.split(source)
    target_b, target_g, target_r = cv2.split(target)
    data, cols, rows = generate_matrix_A(mask)
    b_b = generate_matrix_b(source_b, target_b, mask)
    b_g = generate_matrix_b(source_g, target_g, mask)
    b_r = generate_matrix_b(source_r, target_r, mask)
    # temp = cv2.merge((np.reshape(b_r, (h, w)), np.reshape(b_g, (h, w)), np.reshape(b_b, (h, w))))
    # plt.imshow(temp)
    # plt.show()
    blended_b = solve_sparse_linear_equation(data, cols, rows, b_b, h, w)
    # plt.imshow(blended_b, cmap='gray')
    # plt.show()
    blended_g = solve_sparse_linear_equation(data, cols, rows, b_g, h, w)
    # plt.imshow(blended_g, cmap='gray')
    # plt.show()
    blended_r = solve_sparse_linear_equation(data, cols, rows, b_r, h, w)
    # plt.imshow(blended_r, cmap='gray')
    # plt.show()
    result = cv2.merge((blended_b, blended_g, blended_r))
    # plt.imshow(result)
    # plt.show()
    return result

### `q2.py`
In the main file, after reading images, we scale them to the range [0,1], apply above functions on them and rescaling the result
back to range [0,255]

In [ ]:
import numpy as np
import cv2
import q2_funcs as funcs

img1 = cv2.imread("images/res05.jpg")
img2 = cv2.imread("images/res06.jpg")

img1 = (img1 / 255).astype('float32')
img2 = (img2 / 255).astype('float32')

source = img1[195:485, 355:715, :].astype('float32')
target = img2[195:485, 355:715, :].astype('float32')

mask = np.zeros((290, 360), dtype='float32')
mask[5:-5, 5:-5] = 1

max_matrix, min_matrix = np.ones(source.shape, dtype='float32') * 255, np.zeros(source.shape, dtype='float32')
result = funcs.blend_images(source, target, mask) * 255
result = np.minimum(result, max_matrix)
result = np.maximum(result, min_matrix)
result = result.astype('uint8')
final_result = (img2.copy() * 255).astype('uint8')
final_result[195:485, 355:715, :] = result
cv2.imwrite("images/res07.jpg", final_result)

